<a href="https://colab.research.google.com/github/EbrahimZarei64/CNNHSIE/blob/main/RepVGG_A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import time
import torch.nn as nn
import timm
from repvgg import create_RepVGG_A0

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Creating pre-trained RepVGG model using timm...")
repvgg_model = timm.create_model('repvgg_a0', pretrained=True)
print("Model created successfully.")

Creating pre-trained RepVGG model using timm...
Model created successfully.


In [ ]:
num_features = repvgg_model.head.fc.in_features
repvgg_model.head.fc = nn.Sequential(
    nn.Linear(in_features=num_features, out_features=512),
    nn.Sigmoid()
)
repvgg_model.to(device)
repvgg_model.eval() # Set the model to evaluation mode

print("RepVGG model is ready for inference.")

RepVGG model is ready for inference.


In [ ]:
image = Image.open('./images/lenna-1.tif').convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
print("\nStarting feedforward pass...")
start = time.perf_counter()

with torch.no_grad():
    input_tensor = transform(image).unsqueeze(0).to(device)

    classifier_output = repvgg_model(input_tensor)

    output_array = torch.where(classifier_output > 0.5,
                               torch.ones_like(classifier_output),
                               torch.zeros_like(classifier_output))

execution_time = time.perf_counter() - start
print(f"Execution Time: {execution_time:.6f} seconds")
output_array